https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/ 
ДЗ - оценить возраст человека по изображению. В качестве датасета возьмите датасет только лиц. 
Предположим что сами лица вы находить и извлекать уже умеете. Можете сделать это как продолжение решения по поиску лиц из предыдущих работ.
https://www.google.com/amp/s/www.geeksforgeeks.org/age-detection-using-deep-learning-in-opencv/amp/ пример того как это может работать.  
В качестве фьючеэкстрактора берите любую современную нейронную сеть. Задачу можно решать как задачу регрессии или классификации.  14 14-18 
Если есть время можете попробовать 2 способа.  Попробуйте разблокировать часть слоев после обучения и дообучить модель.


0-14 14-17 18-21 22-


Указание -  что бы решить задачу регресси вам нужно использовать flow_from_dataframe
Указание - для начала вам нужно создать датафрейм в котором будет относительный путь к изображению и целевая метка, потом использовать этот датафрейм при обучении

In [2]:
#!pip install dlib-19.24.99-cp312-cp312-win_amd64.whl
#!pip install face_recognition


In [4]:
import os
from PIL import Image
import face_recognition
import pandas as pd
import tensorflow as tf
import cv2
from scipy import stats
import numpy as np
from tensorflow.keras import regularizers

Вырезание лиц и масштабирование их до разрешения 224*224.

In [ ]:

cur_dir = "./wiki_crop"
prefix = "faces_60"
dir_full_dataset = "./pict_more_60_pixel"
os.makedirs(dir_full_dataset, exist_ok=True)

def process_images(input_folder,output_folder):
    # Путь для сохранения нового датасета
    
    #os.makedirs(output_folder, exist_ok=True)

    for file_name in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file_name)
        try:
            # Загрузка изображения
            image = face_recognition.load_image_file(file_path)
            # Обнаружение лиц
            face_locations = face_recognition.face_locations(image)
            
            for i, face_location in enumerate(face_locations):
                top, right, bottom, left = face_location
                face_image = image[top:bottom, left:right]
                face_pil = Image.fromarray(face_image)

                # Проверка разрешения лица
                if face_pil.width >= 60 and face_pil.height >= 60:
                    # Масштабирование изображения
                    face_resized = face_pil.resize((224, 224))
                    # Сохранение изображения
                    output_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}.jpg")
                    face_resized.save(output_path)
        except Exception as e:
            print(f"Ошибка обработки файла {file_name}: {e}")
dirs = os.listdir(cur_dir)
for dir in dirs:
    process_images(cur_dir+"/"+dir,dir_full_dataset)

print("Обработка завершена!")

Обработка завершена!


Загрузка датасета в 2 массива.

In [5]:
pict_dir = "./pict_more_60_pixel"
correct_files = 0
filepath = []
labels = []
dataset_lenth = 35000
uncorrect_files = 0
files = os.listdir(pict_dir)
for pict in files:
    try:        
        year_of_card = os.path.basename(pict)[-8:-4]
        birth  = os.path.basename(pict)[-19:-15]  
        age =   int(year_of_card)-int(birth)
        if (age>3 and age<105):
            labels.append(int(year_of_card)-int(birth))
            filepath.append(pict_dir+"/"+pict)        
            #image = cv2.imread(pict_dir+"/"+pict)           
            #filepath.append(image)
            correct_files+=1
            if correct_files>=dataset_lenth:
                break
            
        
    except Exception as e:
        uncorrect_files+=1
        print(f"Ошибка обработки файла {pict}: {e}")
        
if len(filepath)==len(labels):
    print("Датасет корректен",len(filepath))
    # склеиваем данные в пандас датасет
    Fseries=pd.Series( filepath, name = 'image' )
    Lseries=pd.Series(labels, name = 'dx')
    data_set = pd.concat([Fseries, Lseries], axis=1)
else:
    print("Датасет некорректен, разные длины массивов меток и данны!!!!!!!!!!!!!!! ")
    
print("Некорректных файлов",uncorrect_files)
print("Корректных файлов",correct_files)
#print(data_set.head)
type(data_set)


Ошибка обработки файла 32339345_1994-11-18._2015.jpg: invalid literal for int() with base 10: '994-'
Ошибка обработки файла 5139000_1963-02-006_2009.jpg: invalid literal for int() with base 10: '963-'
Ошибка обработки файла 706798_1989-08-019_2009.jpg: invalid literal for int() with base 10: '989-'
Датасет корректен 34349
Некорректных файлов 3
Корректных файлов 34349


pandas.core.frame.DataFrame

Обучение модели

In [ ]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Размер картинки для подачи в модели
height   = 224
width    = 224
channels = 3
num_epochs = 5

data_set = data_set[(np.abs(stats.zscore(data_set["dx"])) < 3)]
data_set["dx"].describe()
# Размер пачки для обучения
batch_size = 20
# Размер пачки для валидации
test_batch_size = 50

print("Размер полного датасета",len(data_set))
# Разделяем выборку на обучающую, тестовую и валидационную (случайным образом)

#x_train, x_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)
train_df, test_df = train_test_split (data_set, train_size= .9, shuffle = True, random_state = 42)
valid_df, test_df = train_test_split (test_df, train_size= .5, shuffle = True, random_state = 42)

length    = len(test_df)
# выводим найденное число n
test_steps = int(length/test_batch_size)
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps)


# Создаем генераторы для модели
trgen = ImageDataGenerator()
tvgen = ImageDataGenerator()

# Заполнение генераторов данными:
train_gen = trgen.flow_from_dataframe ( train_df, directory = None, x_col = "image", y_col = "dx", target_size = (height,width), class_mode = 'raw',
                                    color_mode='rgb', shuffle=True, batch_size=batch_size)

# Выборка для тестирования сети после обучения
test_gen = tvgen.flow_from_dataframe ( test_df, directory = None, x_col= "image", y_col = "dx", target_size = (height,width), class_mode = 'raw',
                                    color_mode='rgb', shuffle=False, batch_size=test_batch_size)
# Выборка для тестирования сети во время обучения
valid_gen = tvgen.flow_from_dataframe ( valid_df, directory = None, x_col="image", y_col = "dx", target_size = (height,width), class_mode = 'raw',
                                    color_mode='rgb', shuffle = True, batch_size = batch_size)



# Загрузка предварительно обученной модели EfficientNet-B0
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Замораживание базовых слоёв
base_model.trainable = False

# Добавление новых слоёв
x = base_model.output
x = tf.keras.layers.Dropout(0.3) (x)
x = tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99, epsilon = 0.001 )(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, kernel_regularizer = regularizers.l2(0.02), activity_regularizer = regularizers.l1(0.004),
                bias_regularizer = regularizers.l1(0.006), activation = 'relu')(x)
#x = Dense(128, activation="relu")(x)
output = Dense(1, activation="linear")(x)  # Для регрессии


model = Model(inputs=base_model.input, outputs=output)

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.003), loss="mean_squared_error", metrics=["mae"])

# Callback для остановки при переобучении
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
# Обучение модели
#history = model.fit(    train_df,    validation_data=valid_df,    epochs=num_epochs,    batch_size=32,)

history = model.fit (x = train_gen,  epochs = num_epochs, verbose = "auto",#callbacks = my_callback,
                       validation_data = valid_gen, validation_steps = None,  shuffle = False,  initial_epoch = 0)



test_loss, test_mae = model.evaluate( test_gen, batch_size = test_batch_size, verbose = "auto", steps=test_steps, return_dict = False)[1]*100

print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}")

Размер полного датасета 34106
test batch size:  50   test steps:  34
Found 30695 validated image filenames.
Found 1706 validated image filenames.
Found 1705 validated image filenames.
Epoch 1/5


d:\Soft\ProgramFiles\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1535/1535 ━━━━━━━━━━━━━━━━━━━━ 707s 455ms/step - loss: 270.5476 - mae: 11.4296 - val_loss: 160.0924 - val_mae: 9.1887
Epoch 2/5
1535/1535 ━━━━━━━━━━━━━━━━━━━━ 632s 412ms/step - loss: 158.4678 - mae: 9.0989 - val_loss: 160.5646 - val_mae: 9.6671
Epoch 3/5
1535/1535 ━━━━━━━━━━━━━━━━━━━━ 574s 374ms/step - loss: 151.6763 - mae: 8.9790 - val_loss: 138.3730 - val_mae: 8.5114
Epoch 4/5
1535/1535 ━━━━━━━━━━━━━━━━━━━━ 652s 425ms/step - loss: 150.3801 - mae: 9.0031 - val_loss: 135.6179 - val_mae: 8.5509
Epoch 5/5
1535/1535 ━━━━━━━━━━━━━━━━━━━━ 609s 396ms/step - loss: 145.2046 - mae: 8.8303 - val_loss: 134.3306 - val_mae: 8.5079
34/34 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - loss: 142.7903 - mae: 8.3371


TypeError: cannot unpack non-iterable float object

In [12]:
# Параметры
image_size = (224, 224)
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Загрузка данных из DataFrame
data_set = data_set[(np.abs(stats.zscore(data_set["dx"])) < 3)]
                  

file_names = data_set["image"].values
ages = data_set["dx"].values
data_set["dx"].describe()

count    9944.000000
mean       39.966814
std        16.083669
min         7.000000
25%        27.000000
50%        35.000000
75%        50.000000
max        89.000000
Name: dx, dtype: float64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


# Разделение на train, validate и test
train_files, temp_files, train_ages, temp_ages = train_test_split(file_names, ages, test_size=0.4, random_state=42)
val_files, test_files, val_ages, test_ages = train_test_split(temp_files, temp_ages, test_size=0.5, random_state=42)

# Функция для загрузки изображений по названиям файлов
def load_images(file_names):
    images = []
    for file_name in file_names:
        img = load_img(file_name, target_size=image_size)
        img_array = img_to_array(img) / 255.0  # Нормализация
        images.append(img_array)
    return np.array(images)

# Загрузка изображений
X_train = load_images(train_files)
X_val = load_images(val_files)
X_test = load_images(test_files)

# Загрузка предварительно обученной модели EfficientNet-B0
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
#x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(1, activation="linear")(x)  # Линейный выход для регрессии
model = Model(inputs=base_model.input, outputs=output)


model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mean_squared_error", metrics=["mae","accuracy"])

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Обучение модели
history = model.fit(
    X_train, train_ages,
    validation_data=(X_val, val_ages),
    epochs=num_epochs,
    batch_size=batch_size,
    callbacks=[early_stopping]
)

# Оценка модели на тестовом наборе
test_loss, test_mae, test_accuracy = model.evaluate(X_test, test_ages, batch_size=batch_size)
print(f"Test Loss: {test_loss:.4f}, Test MAE: {test_mae:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 171s 813ms/step - accuracy: 0.0000e+00 - loss: 676.1940 - mae: 20.6068 - val_accuracy: 0.0000e+00 - val_loss: 264.7451 - val_mae: 13.5300
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 141s 754ms/step - accuracy: 0.0000e+00 - loss: 283.4889 - mae: 13.6765 - val_accuracy: 0.0000e+00 - val_loss: 267.2370 - val_mae: 13.9227
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 141s 755ms/step - accuracy: 0.0000e+00 - loss: 278.5885 - mae: 13.5666 - val_accuracy: 0.0000e+00 - val_loss: 264.7198 - val_mae: 13.6343
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 140s 750ms/step - accuracy: 0.0000e+00 - loss: 274.6518 - mae: 13.5122 - val_accuracy: 0.0000e+00 - val_loss: 264.7568 - val_mae: 13.6506
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 142s 759ms/step - accuracy: 0.0000e+00 - loss: 269.3674 - mae: 13.3836 - val_accuracy: 0.0000e+00 - val_loss: 266.1627 - val_mae: 13.8396
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 140s 748ms/step - accuracy: 0.0000e+00 - loss: 275.5171 - mae: 13.6458